# API 資料串接 - 以 Dcard API 實作範例

* 了解 Dcard API 使用方式與回傳內容
* 撰寫程式存取 API 且解析 JSON 格式資料


## 作業目標

* 請利用 API: https://www.dcard.tw/_api/forums/pet/posts?popular=true 回答下列問題：

1. 這個 API 一次會回傳幾筆資料？每一筆資料包含哪些欄位？
2. 取出每一筆資料的「標題」、「貼文時間」、「留言人數」、「按讚人數」
3. 計算熱門/非熱門文章的「平均留言人數」與「平均按讚人數」

In [1]:
# 1. 這個 API 一次會回傳幾筆資料？每一筆資料包含哪些欄位？
import json
import requests
from pprint import pprint

r = requests.get('https://www.dcard.tw/_api/forums/pet/posts?popular=true')
datas = json.loads(r.text)

print(f'一次會回傳 {len(datas)} 筆資料，\n每筆資料包含以下欄位(最上層)：')
pprint(list(datas[0].keys()))

一次會回傳 30 筆資料，
每筆資料包含以下欄位(最上層)：
['id',
 'title',
 'excerpt',
 'anonymousSchool',
 'anonymousDepartment',
 'pinned',
 'forumId',
 'replyId',
 'createdAt',
 'updatedAt',
 'commentCount',
 'likeCount',
 'withNickname',
 'tags',
 'topics',
 'meta',
 'forumName',
 'forumAlias',
 'nsfw',
 'gender',
 'school',
 'replyTitle',
 'mediaMeta',
 'reactions',
 'hidden',
 'customStyle',
 'isSuspiciousAccount',
 'layout',
 'withImages',
 'withVideos',
 'media',
 'reportReasonText',
 'postAvatar']


In [2]:
# 2. 取出每一筆資料的「標題」、「貼文時間」、「留言人數」、「按讚人數」
from datetime import datetime
from datetime import timedelta

for index, data in enumerate(datas):
    try:
        post_time = datetime.strptime(data['createdAt'], '%Y-%m-%dT%H:%M:%S.%fZ')
        post_time = post_time + timedelta(hours=8)  # 轉換時區
    except ValueError:
        post_time = '日期格式錯誤'
    
    print(f"{index+1:2d}. 標題：{data['title']}")
    print(f"    貼文時間：{post_time}")
    print(f"    留言人數：{data['commentCount']}")
    print(f"    按讚人數：{data['likeCount']}\n")

 1. 標題：牠讓女友洗澡變的膽戰心驚
    貼文時間：2020-06-05 11:21:53.715000
    留言人數：166
    按讚人數：2687

 2. 標題：你養的難道是哥吉拉嗎？
    貼文時間：2020-06-05 14:32:56.162000
    留言人數：17
    按讚人數：705

 3. 標題：急！有沒有人有意願收編流浪貓貓
    貼文時間：2020-06-05 13:54:08.988000
    留言人數：13
    按讚人數：147

 4. 標題：兩隻柯基的日常
    貼文時間：2020-06-05 12:27:42.957000
    留言人數：18
    按讚人數：134

 5. 標題：養到各種形象的鼠
    貼文時間：2020-06-05 10:48:54.011000
    留言人數：8
    按讚人數：122

 6. 標題：第二次收服麻雀 失敗
    貼文時間：2020-06-05 15:04:33.009000
    留言人數：17
    按讚人數：34

 7. 標題：尾巴90度的貓
    貼文時間：2020-06-05 12:02:19.277000
    留言人數：4
    按讚人數：32

 8. 標題：寵物客製化商品
    貼文時間：2020-06-05 21:25:44.243000
    留言人數：8
    按讚人數：31

 9. 標題：學校意外捕捉到的可愛畫面，就成了我隨手拍下的紀念照片
    貼文時間：2020-06-05 23:04:07.480000
    留言人數：3
    按讚人數：23

10. 標題：我家喵娘娘很放肆
    貼文時間：2020-06-06 02:30:18.771000
    留言人數：4
    按讚人數：20

11. 標題：可愛的背後
    貼文時間：2020-06-06 00:07:49.122000
    留言人數：5
    按讚人數：19

12. 標題：上廁所的時候一定要這樣偷窺我嗎？
    貼文時間：2020-06-05 16:02:43.394000
    留言人數：0
    按讚人數：17

13. 標題：把我家的貓拿去煎了
    貼文時間：2020-06-05 19:

In [3]:
# 3. 計算熱門/非熱門文章的「平均留言人數」與「平均按讚人數」

# 這邊只抓取並計算前30筆資料
r = requests.get('https://www.dcard.tw/_api/forums/pet/posts?popular=true')
popular_datas = json.loads(r.text)
r = requests.get('https://www.dcard.tw/_api/forums/pet/posts?popular=false')
nonpopular_datas = json.loads(r.text)

popular_comment_count = 0
popular_like_count = 0
nonpopular_comment_count = 0
nonpopular_like_count = 0
for popular_data, nonpopular_data in zip(popular_datas, nonpopular_datas):
    # 如果數值不是整數(錯誤)，則跳過不加
    if type(popular_data['commentCount']) == int:
        popular_comment_count += popular_data['commentCount']
    if type(popular_data['likeCount']) == int:
        popular_like_count += popular_data['likeCount']
    if type(nonpopular_data['commentCount']) == int:
        nonpopular_comment_count += nonpopular_data['commentCount']
    if type(nonpopular_data['likeCount']) == int:
        nonpopular_like_count += nonpopular_data['likeCount']

# 計算平均值
avg_popular_comment = popular_comment_count / len(popular_datas)
avg_popular_like = popular_like_count / len(popular_datas)
avg_nonpopular_comment = nonpopular_comment_count / len(nonpopular_datas)
avg_nonpopular_like = nonpopular_like_count / len(nonpopular_datas)

print(f'[寵物板]   平均留言人數 平均按讚人數')
print(f'熱門文章    {avg_popular_comment:>10.1f}  {avg_popular_like:>10.1f}')
print(f'非熱門文章  {avg_nonpopular_comment:>10.1f}  {avg_nonpopular_like:>10.1f}')

[寵物板]   平均留言人數 平均按讚人數
熱門文章          10.7       135.8
非熱門文章         2.4         3.8
